In [1]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
import tensorflow as tf

(x_train_imdb, y_train_imdb), (x_test_imdb, y_test_imdb) = tf.keras.datasets.imdb.load_data(num_words=1000)

word_index = tf.keras.datasets.imdb.get_word_index()
index2word = dict((i + 3, word) for (word, i) in word_index.items())
index2word[0] = '[pad]'
index2word[1] = '[bos]'
index2word[2] = '[oov]'
x_train_imdb = np.array([' '.join([index2word[idx] for idx in text]) for text in x_train_imdb])
x_test_imdb = np.array([' '.join([index2word[idx] for idx in text]) for text in x_test_imdb])

In [2]:
from tqdm import tqdm

train_doc_length = 0
for doc in tqdm(x_train_imdb):
  tokens = str(doc).split()
  train_doc_length += len(tokens)

print('\nTraining data average document length =', (train_doc_length 
                                                  / len(x_train_imdb)))

100%|██████████| 25000/25000 [00:00<00:00, 26441.90it/s]


Training data average document length = 238.71364


In [3]:
VOCAB_SIZE = 100000
SEQ_MAX_LENGTH = 250
vectorizer = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE,output_mode='int',ngrams=1, name='vector_text',output_sequence_length=SEQ_MAX_LENGTH)
with tf.device('/CPU:0'):
  vectorizer.adapt(x_train_imdb)

In [4]:
vector_model = tf.keras.models.Sequential([
      tf.keras.layers.Input(shape=(1,), dtype=tf.string),
      vectorizer
])
vector_model.predict([['awesome movie']])

1/1 [==============================] - 0s 140ms/step


array([[ 1, 19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 

In [5]:
dummy_embeddings = tf.keras.layers.Embedding(1000, 5)
dummy_embeddings(tf.constant([1, 2, 3])).numpy()

array([[-0.04252988,  0.04213793,  0.00226843, -0.00648656, -0.04334333],
       [-0.0170845 , -0.01579864,  0.00528366, -0.02621027, -0.00614031],
       [ 0.03473804, -0.04966135,  0.00981089, -0.01406177,  0.01161015]],
      dtype=float32)

In [28]:
%pip install fasttext
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
!gzip -d cc.en.300.bin.gz


[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: C:\Users\USER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


'wget' is not recognized as an internal or external command,
operable program or batch file.
'gzip' is not recognized as an internal or external command,
operable program or batch file.


In [34]:
import fasttext 

fasttext_model = fasttext.load_model('cc.en.300.bin')
embedding_matrix = np.zeros(shape=(len(vectorizer.get_vocabulary()), 300))

for i, word in enumerate(vectorizer.get_vocabulary()):
  embedding_matrix[i] = fasttext_model.get_word_vector(word=word)

del fasttext_model

ValueError: cc.en.300.bin cannot be opened for loading!

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image 

def get_fasttext_bigru(num_layers=1, h_size=64):
  inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string, name='txt_input')
  x = vectorizer(inputs)
  x = tf.keras.layers.Embedding(input_dim=len(vectorizer.get_vocabulary()),
                                output_dim=300, name='word_embeddings',
                                trainable=False, weights=[embedding_matrix],
                                mask_zero=True)(x)

  # x = tf.keras.layers.Dense(units=h_size, activation='relu', name='dense')(x)
  x = tf.keras.layers.Dropout(rate=0.25)(x)

  for n in range(num_layers):
    if n != num_layers - 1:
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=h_size, 
                              name=f'bigru_cell_{n}', 
                              return_sequences=True,
                              dropout=0.2))(x)
    else:
      x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(units=h_size, 
                                        name=f'bigru_cell_{n}',
                                        dropout=0.2))(x)

  x = tf.keras.layers.Dropout(rate=0.5)(x)
  o = tf.keras.layers.Dense(units=1, activation='sigmoid', name='lr')(x)
  return tf.keras.models.Model(inputs=inputs, outputs=o, name='simple_rnn')

imdb_fasttext_bigru = get_fasttext_bigru()
print(imdb_fasttext_bigru.summary())

In [ ]:
imdb_fasttext_bigru.compile(loss=tf.keras.losses.BinaryCrossentropy(), 
                            optimizer=tf.keras.optimizers.Adam(),
                            metrics=['binary_accuracy'])
imdb_hist = imdb_fasttext_bigru.fit(x=x_train_imdb, y=y_train_imdb,
                        epochs=10, verbose=1, batch_size=32)

In [ ]:
print(imdb_fasttext_bigru.evaluate(x_test_imdb, y_test_imdb))
predictions = imdb_fasttext_bigru.evaluate(x_test_imdb, y_test_imdb)

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

y = np.array([0.3473,0.3427,0.3338,0.3259,0.3189,0.3095,0.3039,0.2946,0.2860,0.2812])
x = np.array([1,2,3,4,5,6,7,8,9,10])

fig, ax = plt.subplots()
ax.plot(x, y)

ax.grid(True, linestyle='-.')

ax.tick_params(labelcolor='r', labelsize='medium', width=4,)

plt.xlabel("Epochs")
plt.ylabel("Loss")


plt.show()

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(imdb_hist.history).plot(figsize=(8,5))
plt.show